In [ ]:
import os
import pickle
import scipy as sci
import numpy as np
import scipy.stats as stats
# from utills import cal_foof
# from utills import significant_foof
# from utills import signal_plot
# from utills import get_area
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.stats.multicomp as mc
from scipy.signal import periodogram
import mne
import glob
from autoreject import AutoReject
from mne.preprocessing import ICA
import mne_icalabel
from mne_icalabel import label_components
import matplotlib
import torch
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from autoreject import get_rejection_threshold
avgfunc = np.median
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600



In [ ]:

def extract_values(data1, data2):

    regions = ['AllScalp'] # ['Frontal', 'Central', 'Occipital', 'Right', 'Left', 'AllScalp']
    groups = ['parkinson', 'healthy']
    # params = ['real_spec', 'foof_spec', 'curve']
    signals = dict()
    for group in groups:
        signals[group] = dict()
        for region in regions:
            signals[group][region] = dict()
            # for param in params:

            #     signals[group][region][param] = dict()
    aperiodic_mode = 'fixed'
    for region in regions:
        spec_parkinson = []
        foof_parkinson = []
        dterend_parkinson = []
        dterend_healthy = []
        curve_parkinson = []
        spec_healthy = []
        foof_healthy = []
        curve_healthy = []
        for index, subj in enumerate(data1):
            freqs = data1[f'{subj}']['Frontal'].freqs
            real_spec = data1[f'{subj}'][f'{region}'].power_spectrum
            foof = data1[f'{subj}'][f'{region}'].fooofed_spectrum_
            if aperiodic_mode == 'fixed':
                curve = data1[f'{subj}'][f'{region}'].aperiodic_params_[0] \
                        - np.log10(0.00 + freqs ** data1[f'{subj}'][f'{region}'].aperiodic_params_[1])
            else:
                curve = data1[f'{subj}'][f'{region}'].aperiodic_params_[0] \
                        - np.log10(0.00 + freqs ** data1[f'{subj}'][f'{region}'].aperiodic_params_[2])
            spec_parkinson.append(real_spec)
            foof_parkinson.append(foof)
            curve_parkinson.append(curve)
            pd_values = [spec_parkinson, foof_parkinson, curve_parkinson]

        for index, subj in enumerate(data2):
            real_spec_sec = data2[f'{subj}'][f'{region}'].power_spectrum
            foof_sec = data2[f'{subj}'][f'{region}'].fooofed_spectrum_
            if aperiodic_mode == 'fixed':
                curve_sec = data2[f'{subj}'][f'{region}'].aperiodic_params_[0] \
                            - np.log10(0.00 + freqs ** data2[f'{subj}'][f'{region}'].aperiodic_params_[1])
            else:
                curve_sec = data2[f'{subj}'][f'{region}'].aperiodic_params_[0] \
                            - np.log10(0.00 + freqs ** data2[f'{subj}'][f'{region}'].aperiodic_params_[2])
            spec_healthy.append(real_spec_sec)
            foof_healthy.append(foof_sec)
            curve_healthy.append(curve_sec)
            hc_values = [spec_healthy, foof_healthy, curve_healthy]



    return pd_values, hc_values, freqs

In [ ]:
save_path = '/home/hno/datasets'
iowa_dataset = '/home/hno/datasets/update_for_hamzeh/Python/IOWA_DATA/IOWA_Processing/results/'
newmexico_dataset = '/home/hno/datasets/update_for_hamzeh/Python/New_Mexico/Mexico_Data/results/Range_3-40_Hz/'
omega_dataset = '/home/hno/datasets/OMEGA_dataset/fooofed_with_magnetometers/Range_3-40_Hz/'
copen_dataset = '/home/hno/datasets/copen_dataset/fooofed_with_magnetometer/Range_3-40_Hz/'
sandiego_dataset = '/home/hno/datasets/update_for_hamzeh/Python/SanDiego_DATA/SanDiego/result/Range_3-30_Hz/'
turku_dataset = '/home/hno/datasets/update_for_hamzeh/Python/TURKU_DATA/Turku_Data/result/Range_3-40_Hz/'
datasets =  [omega_dataset, copen_dataset, iowa_dataset, newmexico_dataset, turku_dataset]
datasets_names = ['iowa_dataset', 'newmexico_dataset', 'omega_dataset', 'copen_dataset', 'sandiego_dataset', 'turku_dataset']

values = {}
for idx, data in enumerate(datasets):
    # Saving
    with open(data + 'Controls.pkl', 'rb') as f:
         control = pickle.load(f)

    with open(data + 'Parkinson.pkl', 'rb') as f:
        parkinson = pickle.load(f)

    pd_values, hc_values, freq = extract_values(parkinson, control)
    values[idx] = {
        "PD": {"spec": pd_values[0], "foof": pd_values[1], "curve": pd_values[2]},
        "HC": {"spec": hc_values[0], "foof": hc_values[1], "curve": hc_values[2]},
    }

In [ ]:
all_spec_pd_values = [values[name]['PD']['spec'] for name in values]
avg_spec_pd = np.median([np.median(dataset, axis=0) for dataset in all_spec_pd_values], axis=0)
flattened_list = [arr for sublist in all_spec_pd_values for arr in sublist]
all_spec_pd_values = np.array(flattened_list)

all_foof_pd_values = [values[name]['PD']['foof'] for name in values]
flattened_list = [arr for sublist in all_foof_pd_values for arr in sublist]
all_foof_pd_values = np.array(flattened_list)

all_curve_pd_values = [values[name]['PD']['curve'] for name in values]
avg_curve_pd = np.median([np.median(dataset, axis=0) for dataset in all_curve_pd_values], axis=0)
flattened_list = [arr for sublist in all_curve_pd_values for arr in sublist]
all_curve_pd_values = np.array(flattened_list)

all_spec_hc_values = [values[name]['HC']['spec'] for name in values]
avg_spec_hc = np.median([np.median(dataset, axis=0) for dataset in all_spec_hc_values], axis=0)
flattened_list = [arr for sublist in all_spec_hc_values for arr in sublist]
all_spec_hc_values = np.array(flattened_list)

all_foof_hc_values = [values[name]['HC']['foof'] for name in values]
flattened_list = [arr for sublist in all_foof_hc_values for arr in sublist]
all_foof_hc_values = np.array(flattened_list)

all_curve_hc_values = [values[name]['HC']['curve'] for name in values]
avg_curve_hc = np.median([np.median(dataset, axis=0) for dataset in all_curve_hc_values], axis=0)
flattened_list = [arr for sublist in all_curve_hc_values for arr in sublist]
all_curve_hc_values = np.array(flattened_list)

In [ ]:
avg_spec_pd   = np.median(all_spec_pd_values,  axis=0)
avg_spec_hc   = np.median(all_spec_hc_values,  axis=0)
avg_curve_pd  = np.median(all_curve_pd_values, axis=0)
avg_curve_hc  = np.median(all_curve_hc_values, axis=0)

# Compute 95% CI half-widths
ci_spec_pd  = 1.96 * np.std(all_spec_pd_values,  axis=0) / np.sqrt(n_pd)
ci_spec_hc  = 1.96 * np.std(all_spec_hc_values,  axis=0) / np.sqrt(n_hc)
ci_curve_pd = 1.96 * np.std(all_curve_pd_values, axis=0) / np.sqrt(n_pd)
ci_curve_hc = 1.96 * np.std(all_curve_hc_values, axis=0) / np.sqrt(n_hc)

# This block configures all plots to meet .eps format.
mpl.rcParams.update({
    # GUIDELINE: Use a standard, legible sans-serif font.
    "font.family": "sans-serif",
    # Add DejaVu Sans as a fallback to prevent warnings on systems
    # that don't have Helvetica or Arial installed.
    "font.sans-serif": ["Helvetica", "Arial", "DejaVu Sans"],

    # GUIDELINE: Embed all fonts in the output file for portability.
    # Type 42 (TrueType) is the most compatible format.
    "ps.fonttype": 42,
    "pdf.fonttype": 42,

    # GUIDELINE: Set font sizes for legibility (8-12 pt is standard).
    # This creates a clear visual hierarchy.
    "font.size": 13,             # Base size for text
    "axes.titlesize": 13,        # Figure title
    "axes.labelsize": 13,        # X and Y axis labels
    "xtick.labelsize": 13,        # X-axis tick labels
    "ytick.labelsize": 13,        # Y-axis tick labels
    "legend.fontsize": 13,        # Legend text

    # GUIDELINE: Keep line widths clear and visible.
    "lines.linewidth": 1.5,
})


n_pd = len(all_curve_pd_values)
n_hc = len(all_curve_hc_values)


# Figure 1 - Spectrum and Aperiodic Curve (B&W Friendly) ---
fig1, ax1 = plt.subplots(figsize=(6, 4)) # Adjusted for single-column width

# Parkinson's Data (Solid line, dark gray fill)
ax1.plot(freq, avg_spec_pd, label='Parkinson', color='black', linestyle='-')
ax1.fill_between(freq, avg_spec_pd - ci_spec_pd, avg_spec_pd + ci_spec_pd, color='grey', alpha=0.4, rasterized=True)

# Healthy Control Data (Dashed line, light gray fill)
ax1.plot(freq, avg_spec_hc, label='Healthy Control', color='black', linestyle='--')
ax1.fill_between(freq, avg_spec_hc - ci_spec_hc, avg_spec_hc + ci_spec_hc, color='lightgray', alpha=0.5, rasterized=True)

# Aperiodic Fit lines (thinner, dotted lines for distinction)
ax1.plot(freq, avg_curve_pd, linestyle=':', color='black', linewidth=1.0)
ax1.plot(freq, avg_curve_hc, linestyle=':', color='black', linewidth=1.0)


# Labels and aesthetics
# ax1.set_title("A) Power Spectra and Aperiodic Fit")
ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('Power (dB)')
ax1.legend(frameon=False, loc='upper right')
ax1.set_xlim(freq[0], freq[-1])
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
yticks1 = np.linspace(*ax1.get_ylim(), 4)
ax1.set_yticks(np.round(yticks1, 1))
plt.tight_layout()
# GUIDELINE: Save as EPS at 600 DPI for combination art.
plt.savefig(f'{save_path}/EEG_Original_Power_Spectra.eps', dpi=600, format="eps")
plt.savefig(f'{save_path}/EEG_Original_Power_Spectra.png', dpi=1200)
print("Saved Figure 1: Figure1_BW_Power_Spectra.eps (600 DPI)")


# Figure 2 - Adjusted Spectrum (B&W Friendly) ---
dtrnd_pd = avg_spec_pd - avg_curve_pd
dtrnd_hc = avg_spec_hc - avg_curve_hc
ci_dtrnd_pd = 1.96 * np.std(all_spec_pd_values - all_curve_pd_values, axis=0) / np.sqrt(n_pd)
ci_dtrnd_hc = 1.96 * np.std(all_spec_hc_values - all_curve_hc_values, axis=0) / np.sqrt(n_hc)

fig2, ax2 = plt.subplots(figsize=(6, 4))

# Parkinson's Data (Solid line, dark gray fill)
ax2.plot(freq, dtrnd_pd, label='Parkinson', color='black', linestyle='-')
ax2.fill_between(freq, dtrnd_pd - ci_dtrnd_pd, dtrnd_pd + ci_dtrnd_pd, color='grey', alpha=0.4, rasterized=True)

# Healthy Control Data (Dashed line, light gray fill)
ax2.plot(freq, dtrnd_hc, label='Healthy Control', color='black', linestyle='--')
ax2.fill_between(freq, dtrnd_hc - ci_dtrnd_hc, dtrnd_hc + ci_dtrnd_hc, color='lightgray', alpha=0.5, rasterized=True)

# Labels and aesthetics
# ax2.set_title("B) Periodic Power Spectra (Detrended)")
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Adjusted Power (dB)')
ax2.legend(frameon=False, loc='upper right')
ax2.set_xlim(freq[0], freq[-1])
ax2.axhline(0, color='black', linestyle=':', linewidth=0.75)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
plt.tight_layout()
plt.savefig(f'{save_path}/EEG_Adjusted_Power.eps', format="eps")
plt.savefig(f'{save_path}/EEG_Adjusted_Power.png', dpi=1200)
